In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats.stats import pearsonr
from sklearn.metrics import pairwise_distances

In [2]:
userId = 0
movieId = 0
result = pd.DataFrame(columns=['userId','itemId','predict_score'])

In [3]:
Ratings = pd.read_csv('train.csv',sep=',', names = ["userId","itemId","rating"])

In [4]:
Ratings.iloc[388324]

userId    1069
itemId     672
rating       2
Name: 388324, dtype: int64

In [5]:
RatingMat = Ratings.pivot_table(index=['itemId'], columns=['userId'], values=['rating'], fill_value=0)

In [6]:
RatingMat.columns = RatingMat.columns.droplevel()

In [7]:
RatingMat.loc[672,1069] #movieId, userId

2

In [8]:
RatingMat.head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
itemId,,,,,,,,,,,,,,,,,,,,,
1,5,0,0,0,0,4,0,4,5,5,...,0,0,0,0,4,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,2,2,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [9]:
Original_RatingMat = RatingMat.copy()

In [10]:
RatingMat = RatingMat.apply( lambda x: (x[x!=0] - np.sum(x)/np.count_nonzero(x) ) , axis=1)

In [11]:
RatingMat.head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
itemId,,,,,,,,,,,,,,,,,,,,,
1,0.858729,NaN,NaN,NaN,NaN,-0.141271,NaN,-0.141271,0.858729,0.858729,...,NaN,NaN,NaN,NaN,-0.141271,NaN,NaN,NaN,NaN,-1.141271
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.794294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.008791,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.268293,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.731707,-0.731707,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.007194,NaN,NaN,NaN,NaN,NaN


In [12]:
RatingMat = RatingMat.fillna(0.0)

In [13]:
Original_RatingMat_numpy = Original_RatingMat.to_numpy(copy = True)

In [14]:
overall_mean_rating = np.sum(Original_RatingMat_numpy) /np.count_nonzero(Original_RatingMat_numpy)

In [15]:
overall_mean_rating

3.58172626119864

In [16]:
item_similarity = cosine_similarity(RatingMat)

In [17]:
RatingMat.head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
itemId,,,,,,,,,,,,,,,,,,,,,
1,0.858729,0.0,0.0,0.0,0.0,-0.141271,0.0,-0.141271,0.858729,0.858729,...,0.0,0.0,0.0,0.0,-0.141271,0.000000,0.0,0.0,0.0,-1.141271
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.794294,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,-2.008791,0.000000,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.268293,0.000000,0.000000,...,0.0,0.0,0.0,0.0,-0.731707,-0.731707,0.0,0.0,0.0,0.000000
5,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,-2.007194,0.000000,0.0,0.0,0.0,0.000000


In [18]:
test = pd.read_csv('test.csv',sep=',', names = ["userId","itemId"])

In [19]:
i=0
#for i in range():
userId = test.iloc[i]['userId']
itemId = test.iloc[i]['itemId']

In [20]:
userId

1457

In [21]:
rating_bias_of_user = np.zeros( (Original_RatingMat_numpy.shape[1], 1) )

In [22]:
for i in range(Original_RatingMat_numpy.shape[1], 1):
    current_column_user = Original_RatingMat_numpy[:, i]
    current_user_average = np.sum(current_column_user) /np.count_nonzero(current_column_user)
    rating_bias_of_user[i, 0] = current_user_average - overall_mean_rating

In [23]:
print(rating_bias_of_user[1]+1.12)

[1.12]


In [24]:
rating_bias_of_item = np.zeros( (Original_RatingMat_numpy.shape[0], 1) )

for i in range(Original_RatingMat_numpy.shape[0]):
    current_row_item = Original_RatingMat_numpy[i, :]
    current_item_average = np.sum(current_row_item) /np.count_nonzero(current_row_item)
    rating_bias_of_item[i, 0] = current_item_average - overall_mean_rating

In [25]:
lenrating_bias_of_item

NameError: name 'lenrating_bias_of_item' is not defined

In [26]:
w_ij_matrix = np.ones((Original_RatingMat_numpy.shape[0], Original_RatingMat_numpy.shape[1]))

top_k_items=10

#uniformly initialize every column to start with w_ij sums to 1
for i in range(Original_RatingMat_numpy.shape[1]):
    w_ij_matrix[:, i] = w_ij_matrix[:, i] / top_k_items

print(w_ij_matrix)


[[0.1 0.1 0.1 ... 0.1 0.1 0.1]
 [0.1 0.1 0.1 ... 0.1 0.1 0.1]
 [0.1 0.1 0.1 ... 0.1 0.1 0.1]
 ...
 [0.1 0.1 0.1 ... 0.1 0.1 0.1]
 [0.1 0.1 0.1 ... 0.1 0.1 0.1]
 [0.1 0.1 0.1 ... 0.1 0.1 0.1]]


In [27]:
nonzero_indexes = np.where(Original_RatingMat_numpy!=0)

nonzero_indexes_rows = nonzero_indexes[0]

nonzero_indexes_cols = nonzero_indexes[1]

bxj_matrix = np.zeros((Original_RatingMat_numpy.shape[0], Original_RatingMat_numpy.shape[1]))

nnz = len(nonzero_indexes_rows)

for nnz_iter in range(nnz):
    row = nonzero_indexes_rows[nnz_iter]
    col = nonzero_indexes_cols[nnz_iter]
    bxj_matrix[row, col] = overall_mean_rating + rating_bias_of_item[row, 0] + rating_bias_of_user[col, 0]

print(bxj_matrix)

[[4.14127144 0.         0.         ... 0.         0.         4.14127144]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [28]:
col

5997

In [29]:
item_sim_df = pd.DataFrame(item_similarity,index = RatingMat.index,columns=RatingMat.index)

In [30]:
item_sim_df

itemId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
itemId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.074948,0.049321,0.070161,0.036781,0.016162,0.038986,0.023666,-0.008406,0.047545,...,-0.010992,0.000633,0.015878,0.013502,0.026800,0.042889,0.036136,0.016926,0.011617,0.030051
2,0.074948,1.000000,0.029552,0.020047,0.085224,0.025059,0.057651,0.060634,0.070703,0.080576,...,-0.012029,-0.008815,0.015970,0.015301,-0.011486,0.034529,0.006182,-0.013130,-0.006625,-0.001522
3,0.049321,0.029552,1.000000,0.058783,0.134444,0.046669,0.072144,0.034201,0.015972,0.056661,...,0.011236,0.010984,-0.013002,0.009036,0.010720,0.048768,-0.001937,0.027035,0.017836,0.010129
4,0.070161,0.020047,0.058783,1.000000,0.126813,0.001127,0.016936,0.025253,-0.034555,-0.000801,...,0.035430,0.015581,0.010344,0.006965,-0.005836,0.032312,-0.036069,0.011034,0.001734,0.010718
5,0.036781,0.085224,0.134444,0.126813,1.000000,0.017056,0.082604,0.054838,0.054343,0.049838,...,-0.018461,0.041826,0.000324,0.015372,0.029844,0.065444,-0.018858,0.015692,0.000178,-0.016826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.042889,0.034529,0.048768,0.032312,0.065444,0.022550,0.009829,0.030592,-0.003782,0.037442,...,0.019580,0.015172,0.045715,0.040237,0.013420,1.000000,0.030895,0.009257,-0.004206,0.049026
3949,0.036136,0.006182,-0.001937,-0.036069,-0.018858,0.014299,0.001556,-0.045627,0.000584,0.009814,...,0.052678,0.051449,-0.027954,0.024775,0.020773,0.030895,1.000000,0.028287,0.154654,0.048746
3950,0.016926,-0.013130,0.027035,0.011034,0.015692,-0.002270,0.010716,0.077287,-0.007538,0.005852,...,0.010435,0.111214,-0.057013,-0.084479,0.045754,0.009257,0.028287,1.000000,0.041298,0.062119


In [31]:
curr_movie_similarity = pd.DataFrame(item_sim_df.loc[itemId]) #item

In [32]:
Item_Rating = pd.DataFrame(Original_RatingMat.loc[:,userId]) #1 is user 1

In [33]:
Item_Rating.head()

,1457
itemId,
1,4
2,0
3,0
4,0
5,0


In [34]:
curr_movie_similarity = curr_movie_similarity.merge(Item_Rating,right_on='itemId',left_on='itemId')

In [35]:
curr_movie_similarity.rename(columns = {itemId:"similarity",userId:"rating"}, inplace = True) #item

In [36]:
curr_movie_similarity.head()

,similarity,rating
itemId,,
1,-0.020120,4
2,-0.004421,0
3,-0.006999,0
4,-0.018051,0
5,-0.006812,0


In [37]:
top_k_items = 10

In [38]:
baseline estimate = overall_mean_rating + baselin

SyntaxError: invalid syntax (<ipython-input-38-0d94a62e675a>, line 1)

In [39]:
curr_movie_similarity.head()

,similarity,rating
itemId,,
1,-0.020120,4
2,-0.004421,0
3,-0.006999,0
4,-0.018051,0
5,-0.006812,0


In [40]:
curr_movie_similarity = curr_movie_similarity.rename(columns={userId:'rating'})

In [41]:
curr_movie_similarity.head()

,similarity,rating
itemId,,
1,-0.020120,4
2,-0.004421,0
3,-0.006999,0
4,-0.018051,0
5,-0.006812,0


In [42]:
curr_movie_similarity = curr_movie_similarity[curr_movie_similarity.rating != 0]

In [43]:
curr_movie_similarity.head()

,similarity,rating
itemId,,
1,-0.020120,4
16,0.065014,4
21,0.051294,3
34,0.014633,3
39,-0.008309,3


In [44]:
curr_movie_similarity= curr_movie_similarity.sort_values(by='similarity',ascending=False)

In [45]:
top_k_plus_one_item_similarity = curr_movie_similarity.nlargest(n = top_k_items, columns = "similarity")

In [47]:
top_k_plus_one_item_similarity

,similarity,rating
itemId,,
1351,0.122718,4
1095,0.116185,3
2289,0.115172,5
3129,0.106124,4
3108,0.104651,3
1250,0.099289,5
58,0.096526,5
2064,0.094950,5
3468,0.094816,4


In [48]:
score = (top_k_plus_one_item_similarity['similarity'] * top_k_plus_one_item_similarity['rating']).sum()
/(top_k_plus_one_item_similarity['similarity'].sum())

In [49]:
score

4.091564640369786

In [49]:
if score == 0: score = overall_mean_rating

In [50]:
result = result.append({'userId': userId,'itemId': movieId, 'predict_score': score }, ignore_index=True)

In [51]:
result

,userId,itemId,predict_score
0,1457.0,1245.0,4.091565


In [53]:
result.to_csv('submit.csv',header=False,index=False)

In [54]:
overall_mean_rating

3.58172626119864